# Example notebook to use the GloUrbEE extraction tool

First, you need to create a Google Cloud Project with the EarthEngine API enabled. 
Then create a service account with the EarthEngine Viewer role, and download a login json key for this service account. 

## Login to Google Cloud

The next cell load Earth Engine package and login to the API using the service account.

In [ ]:
import ee

credentials = ee.ServiceAccountCredentials('samuel@ee-glourb.iam.gserviceaccount.com', './earthengine-key.json')
ee.Initialize(credentials)

# Set your Google Cloud EarthEngine-enabled project name (for asset exports)
ee_project_name='ee-glourb'

## Import the required packages

In [ ]:
from glourbee import (
    assets_management,
    workflow,
    collection
)

# The next lines are useful if you make modifications in the module files and want to reload it without restarting the notebook kernel
import importlib
importlib.reload(assets_management)
importlib.reload(workflow)
importlib.reload(collection)

## 1. Extraction zones management

### Use your own extraction zones

In [ ]:
# Define your extraction zones
extraction_zones = assets_management.ExtractionZones(local_file='./example_data/Lhasa_RC_DGO2km_updated.shp', # Local file containing extraction features
                                                     fid_field='DGO_FID', # Unique identifier field name
                                                     zone_type='DGOs', # Literal description of the zones types (eg. DGOs, Cites, Gravel quarries, ...)
                                                     description='DGOs of 2km for Lhasa river', # Literal description of the zones. Give a maximum details so other GloUrb researcher can understand chat are these zones.
                                                     author='Georges Abitbol' # Identify the producer of those extraction zones
                                                    )

In [ ]:
# Check your assigned UUID, you can use it to retrieve your zones the next time you will start GloUrbEE
zones_uuid = extraction_zones.asset_uuid
print(zones_uuid)

In [ ]:
# Replace this uuid with the output of the previous cell to keep trace of your current work
zones_uuid = '423023412407495ba4d441e835ad0f19'

In [ ]:
# Check if your zones are already uploaded to GEE
extraction_zones.gee_state # 'none', 'partial' or 'complete'

In [ ]:
# Upload the zones to GEE
extraction_zones.upload_to_gee()

In [ ]:
# Check again if your zones are already uploaded to GEE
extraction_zones.update_gee_state()
extraction_zones.gee_state # 'none', 'partial' or 'complete'

In [ ]:
# Once the upload process is started, you can retrieve your asset with it's UUID
extraction_zones = assets_management.ExtractionZones(asset_uuid=zones_uuid)

In [ ]:
# More details on upload tasks
extraction_zones.update_gee_state()
extraction_zones.linked_tasks

In [ ]:
# You can use this to wait for upload to complete
extraction_zones.wait_for_tasks()

In [ ]:
# You can cancel upload tasks if needed
extraction_zones.cancel_linked_tasks()

### Use other extraction zones already uploaded to the GEE project

You can explore other dataset created by GloUrb researchers and uploaded to the GEE project with the two following methods.

In [ ]:
# Get all the extraction zones available on the GEE project
glourbee_zones = collection.getGlourbeeExtractionZones()
glourbee_zones

In [ ]:
# Use one of the extraction zones for your current work
extraction_zones = assets_management.ExtractionZones(asset_uuid='423023412407495ba4d441e835ad0f19')

In [ ]:
# Get all the metrics datasets already calculated for an extraction zone dataset
glourbee_metrics = collection.getGlourbeeMetrics(zones_uuid='8e57f5112a314a68a39666993ec1f773')
glourbee_metrics

In [ ]:
# Download one of the metrics datasets
metrics = assets_management.MetricsDataset(asset_uuid='54a79deaaf47443392b9ae5178dd5f03', parent_zones='8e57f5112a314a68a39666993ec1f773')
metrics.download(output_file='./example_data/metrics.csv',
                 overwrite=False # By setting this to False, download will resume at his previous step if it was interrupted
                )

### Delete extraction zones and associated metrics from the GEE project

In [ ]:
# WARNING: unreversible operation
extraction_zones.delete()

## 2. Indicators workflow

It is possible to extract indicators based on the JRC Global Surface Water Mapping Layers

In [ ]:
workflow.indicatorsWorkflow(extraction_zones=extraction_zones, 
                            output_csv='./example_data/jrc_indicators.csv')

## 3. Metrics workflow

In [ ]:
# Start a new matrics calculation
metrics = workflow.startWorkflow(zones_dataset=extraction_zones, # Your extraction zones dataset
                                 satellite_type='Landsat', # Landsat | Sentinel-2
                                 start='1990-01-01',
                                 end='1990-01-31',
                                 cloud_filter=80, # Maximum cloud coverage for an image
                                 cloud_masking=True, # Masking the clouds on the metrics results (recommended)
                                 mosaic_same_day=True, # Merge all images of same day into one

                                 # With those three last parameters, you can customize the way to define each mask created to calculate metrics. Check README for more details
                                 watermask_expression=None,
                                 activechannel_expression=None,
                                 vegetation_expression=None
                                )

In [ ]:
# Check your assigned UUID, you can use it to retrieve your metrics the next time you will start GloUrbEE
metrics_uuid = metrics.asset_uuid
print(metrics_uuid)

In [ ]:
# Replace this uuid with the output of the previous cell to keep trace of your current work
metrics_uuid = 'b2ed1f33cec1493099d557aeff34812c'

In [ ]:
# Retrieve metrics dataset with UUID
metrics = assets_management.MetricsDataset(parent_zones=extraction_zones,
                                           asset_uuid=metrics_uuid)

In [ ]:
# Check your metrics calculation state
metrics.update_gee_state()
metrics.gee_state

In [ ]:
# More details on calculation tasks
metrics.update_gee_state()
metrics.linked_tasks

In [ ]:
# Cancel calculation tasks if needed
metrics.cancel_linked_tasks()

In [ ]:
# Delete the metrics dataset from GEE
# WARNING: non reversible operation
metrics.delete()

### Export results
When all the computation tasks are complete, use the follwing to merge the result into one file and download it locally.

In [ ]:
metrics.download(output_file='./example_data/metrics.csv',
                 overwrite=False # By setting this to False, download will resume at his previous step if it was interrupted
                )